In [3]:
# adaptive svm
import math
import numpy as np 
import pandas as pd 

# Note, this is for binary classification only
def kernel(kernel='linear', gamma = 0.01, r=1/2, d=2):
	'''
		gamma : 1/2*sigma**2 for the gaussian kernel
		r : the bias for polynomial kernel
		d : the power base for the polynomial kernel
	'''

	# the default linear kernel
	operation = lambda x, y : np.dot(x, y)

	if(kernel == 'rbf'):
		operation = lambda x, y : -gamma*np.linalg.norm(x-y)
	if(kernel == 'poly'):
		operation = lambda x, y : (np.dot(x,y) + r) ** d 

	return operation

# mse loss
def loss(predictions, labels):
	if(predictions.shape[0] != labels.shape[0]):
		print("[INFO] Labels and Predictions are not of the same shape ... ")
		return None
	else:
		# loop thru pairs of prediction and label
		loss = 0
		for j in range(predictions.shape[0]): # or labels.shape[0]
			loss += (predictions[j] - labels[j]) ** 2

		return math.sqrt(loss)

class_1 = np.array([[2,3], [1,2], [2.5, 3.5],
                    [1.5, 2.5], [2,2], [2.5,2.5],
                    [1.5,3],[2.5,1],[1,1]])

class_2 = np.array([[4,5],[5,5],[4,6],
                    [4.5,5.5],[5.5,5.5],[6,6],
                    [5,4],[5,6],[6,5]])

# validation data
class_1_ = np.array([[1.2,2.1],[2.1,2],[1.3,1.7]])
class_2_ = np.array([[4,4],[3.8,5.4],[4.2,3.4]])

x = np.concatenate((class_1, class_2))
x_ = np.concatenate((class_1_, class_2_))
y = np.array([1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1])

# let's go a little bit object oriented shall we
class KernelSVM:
	def __init__(self):
		self.x = None 
		self.y = None 
		self.w = None
		self.kernel = None

	def fit(self,x, y, iterations=100000, alpha=0.001, l=0.01):
		'''
			- iterations is the max number of training iterations
			- alpha is the learning rate
			- l is the regularization parameter
		'''

		# just create a copy for inference purpose
		self.x = x
		self.y = y

		bias = 10.0

		if(not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)):
			print("[INFO] The input and output must be numpy array ... ")
			return None
		else:
			# check if the input is of right dimension
			if(len(x.shape) != 2 or len(y.shape) != 1):
				print("[INFO] Input and output are of wrong shape ...")
				return None
			else:
				# svm is optimized based on hinge loss
				# hinge(x,w) = Max(0, 1 - y * <x,w>)
				# we need to optimize 2/||w|| -> the regularized loss is 
				# L = summation(Max(0, 1 - y * <x,w>)) + ||w||**2/2
				K = kernel(kernel='rbf')
				self.kernel = K

				# first, calculate all f_i vectors from the kernel
				dataset_size = x.shape[0]
				f_i = np.zeros((dataset_size, dataset_size))

				# construct f_i which is a matrix of kernelized inputs
				for i in range(dataset_size):
					f = np.zeros((dataset_size,))
					for j in range(dataset_size):
						f[j] = K(x[i], x[j])

					f_i[i] = f 

				# print(f_i)

				# now all we have to do is applying svm on f_1 and y
				self.w = np.ones((dataset_size, ), dtype=np.float32) # a vector with same size as f_i input vectors

				# loop thru the training iterations
				previous_loss = 0
				for i in range(iterations) :
					predictions = np.zeros((y.shape[0], ))
					# loop thru all the data
					for j in range(f_i.shape[0]):
						f_i_ = f_i[j]
						prediction = np.dot(self.w, f_i_) + bias 
						predictions[j] = prediction

						# check if this is a mis-classification case
						if(1 - y[j] * (np.dot(self.w, f_i_) + bias) > 0): # misclassification
							# loop thru the elements in the weight vector
							for i_ in range(self.w.shape[0]):
								self.w[i_] = self.w[i_] - alpha * (-y[j] * f_i_[i_] + l * self.w[i_])

							bias = alpha - y[j] * alpha

						else: # no misclassification occured
							# loop thru the elements of the weight vector again 
							for i_ in range(self.w.shape[0]):
								self.w[i_] = self.w[i_] - alpha * (l*self.w[i_])

					mse = loss(predictions, y)

					if(mse > previous_loss and i != 0):
						break

					# if the reduction in loss barely matters
					# we break the process
					if(previous_loss - mse < 1e-8 and i > 1000):
						break

					previous_loss = mse
					print("[INFO] Epoch : " + str(i+1) + " | Loss = " + str(mse))

	def predict(self, x):
		predictions = []

		# for each of the new data
		for i in range(x.shape[0]):
			# loop thru the training dataset
			prediction = 0
			for j in range(self.x.shape[0]):
				prediction += self.y[j] * self.w[j] * self.kernel(x[i], self.x[j])

			print(prediction)
			if(prediction > 0):
				predictions.append(1)
			else:
				predictions.append(-1)

		return predictions 


svm = KernelSVM()
svm.fit(x,y)

predictions = svm.predict(x_)
print(predictions)
# time to make prediction to see if it is true


[INFO] Epoch : 1 | Loss = 27.59708029838543
[INFO] Epoch : 2 | Loss = 4.786938833689266
[INFO] Epoch : 3 | Loss = 4.786408135065274
[INFO] Epoch : 4 | Loss = 4.785877593701372
[INFO] Epoch : 5 | Loss = 4.785347158980403
[INFO] Epoch : 6 | Loss = 4.784816778625111
[INFO] Epoch : 7 | Loss = 4.784286433110835
[INFO] Epoch : 8 | Loss = 4.783756215389325
[INFO] Epoch : 9 | Loss = 4.783226171234549
[INFO] Epoch : 10 | Loss = 4.782696302906904
[INFO] Epoch : 11 | Loss = 4.78216649217533
[INFO] Epoch : 12 | Loss = 4.781636733033487
[INFO] Epoch : 13 | Loss = 4.7811070652619705
[INFO] Epoch : 14 | Loss = 4.780577516817279
[INFO] Epoch : 15 | Loss = 4.780048069080691
[INFO] Epoch : 16 | Loss = 4.779518703359856
[INFO] Epoch : 17 | Loss = 4.77898940817771
[INFO] Epoch : 18 | Loss = 4.77846023372309
[INFO] Epoch : 19 | Loss = 4.77793114167127
[INFO] Epoch : 20 | Loss = 4.7774021892559615
[INFO] Epoch : 21 | Loss = 4.776873422694981
[INFO] Epoch : 22 | Loss = 4.776344729952373
[INFO] Epoch : 23 | L

[INFO] Epoch : 213 | Loss = 4.677226834781673
[INFO] Epoch : 214 | Loss = 4.676717620036112
[INFO] Epoch : 215 | Loss = 4.676208525810788
[INFO] Epoch : 216 | Loss = 4.675699493768334
[INFO] Epoch : 217 | Loss = 4.675190556717968
[INFO] Epoch : 218 | Loss = 4.674681679939739
[INFO] Epoch : 219 | Loss = 4.674172913483309
[INFO] Epoch : 220 | Loss = 4.673664249568188
[INFO] Epoch : 221 | Loss = 4.6731556840894015
[INFO] Epoch : 222 | Loss = 4.672647176317404
[INFO] Epoch : 223 | Loss = 4.67213873625212
[INFO] Epoch : 224 | Loss = 4.671630390743326
[INFO] Epoch : 225 | Loss = 4.671122229667198
[INFO] Epoch : 226 | Loss = 4.670614185588
[INFO] Epoch : 227 | Loss = 4.670106222738126
[INFO] Epoch : 228 | Loss = 4.669598325245375
[INFO] Epoch : 229 | Loss = 4.669090578924087
[INFO] Epoch : 230 | Loss = 4.6685828987869495
[INFO] Epoch : 231 | Loss = 4.668075306893971
[INFO] Epoch : 232 | Loss = 4.667567837250806
[INFO] Epoch : 233 | Loss = 4.667060454373743
[INFO] Epoch : 234 | Loss = 4.666553

[INFO] Epoch : 406 | Loss = 4.58076195359612
[INFO] Epoch : 407 | Loss = 4.580271572839553
[INFO] Epoch : 408 | Loss = 4.5797812836683995
[INFO] Epoch : 409 | Loss = 4.57929110636146
[INFO] Epoch : 410 | Loss = 4.5788010067678675
[INFO] Epoch : 411 | Loss = 4.578310980457634
[INFO] Epoch : 412 | Loss = 4.577821035498411
[INFO] Epoch : 413 | Loss = 4.577331209652367
[INFO] Epoch : 414 | Loss = 4.5768414880394666
[INFO] Epoch : 415 | Loss = 4.576351848957063
[INFO] Epoch : 416 | Loss = 4.575862336135922
[INFO] Epoch : 417 | Loss = 4.575372939639175
[INFO] Epoch : 418 | Loss = 4.574883633185859
[INFO] Epoch : 419 | Loss = 4.574394403673591
[INFO] Epoch : 420 | Loss = 4.573905294597093
[INFO] Epoch : 421 | Loss = 4.5734162420111195
[INFO] Epoch : 422 | Loss = 4.57292728972659
[INFO] Epoch : 423 | Loss = 4.572438448746436
[INFO] Epoch : 424 | Loss = 4.57194966469057
[INFO] Epoch : 425 | Loss = 4.571461017392251
[INFO] Epoch : 426 | Loss = 4.570972473165371
[INFO] Epoch : 427 | Loss = 4.5704

[INFO] Epoch : 588 | Loss = 4.4930779186695515
[INFO] Epoch : 589 | Loss = 4.4926047401765175
[INFO] Epoch : 590 | Loss = 4.492131676809756
[INFO] Epoch : 591 | Loss = 4.491658685829908
[INFO] Epoch : 592 | Loss = 4.491185779834829
[INFO] Epoch : 593 | Loss = 4.490712924205121
[INFO] Epoch : 594 | Loss = 4.490240193628789
[INFO] Epoch : 595 | Loss = 4.489767571638411
[INFO] Epoch : 596 | Loss = 4.489295074466246
[INFO] Epoch : 597 | Loss = 4.488822636082444
[INFO] Epoch : 598 | Loss = 4.488350285298323
[INFO] Epoch : 599 | Loss = 4.487878055993775
[INFO] Epoch : 600 | Loss = 4.48740590991501
[INFO] Epoch : 601 | Loss = 4.486933900402705
[INFO] Epoch : 602 | Loss = 4.486461913596688
[INFO] Epoch : 603 | Loss = 4.485990038920565
[INFO] Epoch : 604 | Loss = 4.485518243098911
[INFO] Epoch : 605 | Loss = 4.485046527254982
[INFO] Epoch : 606 | Loss = 4.484574880177819
[INFO] Epoch : 607 | Loss = 4.484103371123888
[INFO] Epoch : 608 | Loss = 4.483631969991498
[INFO] Epoch : 609 | Loss = 4.483

[INFO] Epoch : 778 | Loss = 4.404824757103605
[INFO] Epoch : 779 | Loss = 4.404368943375244
[INFO] Epoch : 780 | Loss = 4.40391321827427
[INFO] Epoch : 781 | Loss = 4.403457577643402
[INFO] Epoch : 782 | Loss = 4.403001982208181
[INFO] Epoch : 783 | Loss = 4.402546459529097
[INFO] Epoch : 784 | Loss = 4.4020910500875905
[INFO] Epoch : 785 | Loss = 4.401635794268797
[INFO] Epoch : 786 | Loss = 4.4011805933703
[INFO] Epoch : 787 | Loss = 4.4007255289935445
[INFO] Epoch : 788 | Loss = 4.400270555238753
[INFO] Epoch : 789 | Loss = 4.399815621979852
[INFO] Epoch : 790 | Loss = 4.39936076446451
[INFO] Epoch : 791 | Loss = 4.398905984567685
[INFO] Epoch : 792 | Loss = 4.398451327392869
[INFO] Epoch : 793 | Loss = 4.3979967821248325
[INFO] Epoch : 794 | Loss = 4.39754233537452
[INFO] Epoch : 795 | Loss = 4.397087946624205
[INFO] Epoch : 796 | Loss = 4.396633671134083
[INFO] Epoch : 797 | Loss = 4.396179452621285
[INFO] Epoch : 798 | Loss = 4.395725332931441
[INFO] Epoch : 799 | Loss = 4.395271

...

[INFO] Epoch : 48269 | Loss = 2.135168730885474
[INFO] Epoch : 48270 | Loss = 2.135168695740726
[INFO] Epoch : 48271 | Loss = 2.1351686605959794
[INFO] Epoch : 48272 | Loss = 2.1351686254512363
[INFO] Epoch : 48273 | Loss = 2.1351685903064945
[INFO] Epoch : 48274 | Loss = 2.135168555161755
[INFO] Epoch : 48275 | Loss = 2.135168520017018
[INFO] Epoch : 48276 | Loss = 2.135168484872283
[INFO] Epoch : 48277 | Loss = 2.13516844972755
[INFO] Epoch : 48278 | Loss = 2.1351684145828194
[INFO] Epoch : 48279 | Loss = 2.1351683794380913
[INFO] Epoch : 48280 | Loss = 2.135168344293364
[INFO] Epoch : 48281 | Loss = 2.13516830914864
[INFO] Epoch : 48282 | Loss = 2.1351682740039184
[INFO] Epoch : 48283 | Loss = 2.1351682388591984
[INFO] Epoch : 48284 | Loss = 2.1351682037144806
[INFO] Epoch : 48285 | Loss = 2.1351681685697654
[INFO] Epoch : 48286 | Loss = 2.135168133425052
[INFO] Epoch : 48287 | Loss = 2.135168098280341
[INFO] Epoch : 48288 | Loss = 2.135168063135632
[INFO] Epoch : 48289 | Loss = 2.1

[INFO] Epoch : 48511 | Loss = 2.1351631781116365
[INFO] Epoch : 48512 | Loss = 2.1351631622656897
[INFO] Epoch : 48513 | Loss = 2.135163146419744
[INFO] Epoch : 48514 | Loss = 2.1351631305737984
[INFO] Epoch : 48515 | Loss = 2.1351631147278534
[INFO] Epoch : 48516 | Loss = 2.135163098881909
[INFO] Epoch : 48517 | Loss = 2.1351630830359642
[INFO] Epoch : 48518 | Loss = 2.135163067190021
[INFO] Epoch : 48519 | Loss = 2.135163051344078
[INFO] Epoch : 48520 | Loss = 2.135163035498135
[INFO] Epoch : 48521 | Loss = 2.1351630196521927
[INFO] Epoch : 48522 | Loss = 2.135163003806251
[INFO] Epoch : 48523 | Loss = 2.1351629879603093
[INFO] Epoch : 48524 | Loss = 2.135162972114369
[INFO] Epoch : 48525 | Loss = 2.135162956268428
[INFO] Epoch : 48526 | Loss = 2.135162940422488
[INFO] Epoch : 48527 | Loss = 2.1351629245765484
[INFO] Epoch : 48528 | Loss = 2.135162908730609
[INFO] Epoch : 48529 | Loss = 2.1351628928846704
[INFO] Epoch : 48530 | Loss = 2.1351628770387325
[INFO] Epoch : 48531 | Loss = 

[INFO] Epoch : 48747 | Loss = 2.1351594384808643
[INFO] Epoch : 48748 | Loss = 2.135159422635025
[INFO] Epoch : 48749 | Loss = 2.135159406789186
[INFO] Epoch : 48750 | Loss = 2.1351593909433477
[INFO] Epoch : 48751 | Loss = 2.1351593750975097
[INFO] Epoch : 48752 | Loss = 2.135159359251672
[INFO] Epoch : 48753 | Loss = 2.135159343405835
[INFO] Epoch : 48754 | Loss = 2.135159327559999
[INFO] Epoch : 48755 | Loss = 2.1351593117141623
[INFO] Epoch : 48756 | Loss = 2.1351592958683265
[INFO] Epoch : 48757 | Loss = 2.1351592800224912
[INFO] Epoch : 48758 | Loss = 2.1351592641766564
[INFO] Epoch : 48759 | Loss = 2.135159248330822
[INFO] Epoch : 48760 | Loss = 2.1351592324849884
[INFO] Epoch : 48761 | Loss = 2.135159216639155
[INFO] Epoch : 48762 | Loss = 2.1351592007933218
[INFO] Epoch : 48763 | Loss = 2.135159184947489
[INFO] Epoch : 48764 | Loss = 2.135159169101657
[INFO] Epoch : 48765 | Loss = 2.1351591532558256
[INFO] Epoch : 48766 | Loss = 2.135159137409994
[INFO] Epoch : 48767 | Loss = 

[INFO] Epoch : 48989 | Loss = 2.135155603800963
[INFO] Epoch : 48990 | Loss = 2.135155587955233
[INFO] Epoch : 48991 | Loss = 2.1351555721095044
[INFO] Epoch : 48992 | Loss = 2.1351555562637756
[INFO] Epoch : 48993 | Loss = 2.135155540418047
[INFO] Epoch : 48994 | Loss = 2.135155524572319
[INFO] Epoch : 48995 | Loss = 2.1351555087265917
[INFO] Epoch : 48996 | Loss = 2.1351554928808647
[INFO] Epoch : 48997 | Loss = 2.1351554770351386
[INFO] Epoch : 48998 | Loss = 2.135155461189412
[INFO] Epoch : 48999 | Loss = 2.135155445343687
[INFO] Epoch : 49000 | Loss = 2.1351554294979618
[INFO] Epoch : 49001 | Loss = 2.1351554136522375
[INFO] Epoch : 49002 | Loss = 2.135155397806513
[INFO] Epoch : 49003 | Loss = 2.1351553819607894
[INFO] Epoch : 49004 | Loss = 2.135155366115066
[INFO] Epoch : 49005 | Loss = 2.135155350269343
[INFO] Epoch : 49006 | Loss = 2.135155334423621
[INFO] Epoch : 49007 | Loss = 2.1351553185778993
[INFO] Epoch : 49008 | Loss = 2.1351553027321777
[INFO] Epoch : 49009 | Loss = 

[INFO] Epoch : 49231 | Loss = 2.135151769147619
[INFO] Epoch : 49232 | Loss = 2.1351517533019986
[INFO] Epoch : 49233 | Loss = 2.1351517374563795
[INFO] Epoch : 49234 | Loss = 2.1351517216107605
[INFO] Epoch : 49235 | Loss = 2.135151705765142
[INFO] Epoch : 49236 | Loss = 2.135151689919524
[INFO] Epoch : 49237 | Loss = 2.1351516740739065
[INFO] Epoch : 49238 | Loss = 2.1351516582282892
[INFO] Epoch : 49239 | Loss = 2.1351516423826724
[INFO] Epoch : 49240 | Loss = 2.135151626537056
[INFO] Epoch : 49241 | Loss = 2.1351516106914405
[INFO] Epoch : 49242 | Loss = 2.135151594845825
[INFO] Epoch : 49243 | Loss = 2.13515157900021
[INFO] Epoch : 49244 | Loss = 2.135151563154596
[INFO] Epoch : 49245 | Loss = 2.1351515473089817
[INFO] Epoch : 49246 | Loss = 2.1351515314633684
[INFO] Epoch : 49247 | Loss = 2.135151515617755
[INFO] Epoch : 49248 | Loss = 2.135151499772143
[INFO] Epoch : 49249 | Loss = 2.1351514839265304
[INFO] Epoch : 49250 | Loss = 2.1351514680809185
[INFO] Epoch : 49251 | Loss = 

In [4]:
svm.w

array([ 1.4126302,  2.028444 ,  0.8460581,  1.8088175,  1.7873739,
        1.3795335,  1.5867043,  1.7191513,  2.1473951, -1.1308479,
       -1.6523802, -1.4921174, -1.6341459, -1.9827992, -2.1351416,
       -1.0856599, -1.9462202, -1.879334 ], dtype=float32)